# Breakout Group 4, Application: Disambiguation

# Create Peripleo web map from disambiguated data

In order to demonstrate the potential of data produced by disambiguation processes this notebook builds a pipeline for creating an instance of the Peripleo web map interface.

### Input
It takes an input of a spreadsheet of disambiguated data similar to the output of the process group. This must contain place data and contain metadata for Object ID, Object Title, associated Wikidata ID.

### Output
It creates an output of a URL for a publicly available instance of Peripleo web map.

### Steps
In between it offers the following steps:

* Query Wikidata to obtain coordinate data for each object (row)
* Generate GeoJSON in the Linked Places format that Peripleo uses
* Make data available on Github Pages in a cloned Peripleo repository (https://github.com/britishlibrary/peripleo)




In [ ]:
#@title Install Dependencies

!pip install folium SPARQLWrapper geojson PyGithub geopandas fiona


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 19.0 MB/s eta 0:00:00


In [ ]:
#@title 1) Query Wikidata to obtain coordinate data
#@markdown Spreadsheet should contain entity_label, entity_text, wikidata_uri .
#@markdown Enter a URL for your data in the box below:

import pandas as pd
import time # Import the time module

URL = "http://145.38.185.232/enriching/disambiguation_annotation.csv" #@param {type:'string'}

#@title Function to fetch from Wikidata

from SPARQLWrapper import SPARQLWrapper, JSON

def query_wikidata(uri, endpoint="https://query.wikidata.org/sparql", cache=dict()):

    if uri in cache:
        return cache[uri]

    q = """
    SELECT DISTINCT ?uri ?uriLabel ?uriDescription ?latitude ?longitude WHERE {
        ?uri wdt:P31|wdt:P279 [] .

        OPTIONAL {
            ?uri p:P625 ?coordinate.
            ?coordinate ps:P625 ?coord.
            ?coordinate psv:P625 ?coordinate_node.
            ?coordinate_node wikibase:geoLongitude ?longitude.
            ?coordinate_node wikibase:geoLatitude ?latitude.
            }

        VALUES ?uri { <URIHIER> }

        SERVICE wikibase:label { bd:serviceParam wikibase:language "en,nl,de,fr,it,es,". }
    }
    """.replace(
        "URIHIER", uri
    )

    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(q)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    if len(results["results"]["bindings"]) == 0:
        return "", "", "", ""

    label = results["results"]["bindings"][0]["uriLabel"]["value"]
    description = (
        results["results"]["bindings"][0].get("uriDescription", {}).get("value")
    )
    latitude = results["results"]["bindings"][0].get("latitude", {}).get("value")
    longitude = results["results"]["bindings"][0].get("longitude", {}).get("value")

    cache[uri] = label, description, latitude, longitude
    return label, description, latitude, longitude

#Test Wikidata Query
# cache = dict()
# uri = "http://www.wikidata.org/entity/Q43631"

# label, description, latitude, longitude = query_wikidata(uri, cache=cache)

# print(f"Label: {label}")
# print(f"Description: {description}")
# print(f"Latitude: {latitude}")
# print(f"Longitude: {longitude}")


def enrich_df(df: pd.DataFrame) -> pd.DataFrame:

  cache = dict()

  for i, row in df.iterrows():

    if row.entity_label != "LOC":
      continue
    elif pd.isna(row.wikidata_uri):
      continue

    if '/wiki/' in row.wikidata_uri:
      uri = row.wikidata_uri.replace('https://www.wikidata.org/wiki/', 'http://www.wikidata.org/entity/')
      df.loc[i, "wikidata_uri"] = uri
    else:
      uri = row.wikidata_uri

    label, description, latitude, longitude = query_wikidata(uri, cache=cache)

    df.loc[i, "wikidata_label"] = label
    df.loc[i, "wikidata_description"] = description
    df.loc[i, "wikidata_latitude"] = latitude
    df.loc[i, "wikidata_longitude"] = longitude

    time.sleep(0.5) # Add a small delay between queries

  return df

df = pd.read_csv(URL)
df.head()

df = enrich_df(df)
df[df['entity_label'] == 'LOC'].head()

,Unique ID,line_nbr,entity_label,entity_text,dbpedia_uri,wikidata_uri,wikidata_label,wikidata_description,wikidata_latitude,wikidata_longitude
9,P_SL-5275-71,5,LOC,Surinam,http://dbpedia.org/resource/Suriname,http://www.wikidata.org/entity/Q730,Suriname,country in South America,4.0,-56.0
10,P_Q-4-295,6,LOC,Augsburg,http://dbpedia.org/resource/Augsburg,http://www.wikidata.org/entity/Q2749,Augsburg,"city in Bavaria, Germany",48.368888888889,10.897777777778
11,P_Q-4-295,6,LOC,Red Tower,NaN,NaN,NaN,NaN,NaN,NaN
12,P_Q-4-295,6,LOC,Rotes Tor,NaN,NaN,NaN,NaN,NaN,NaN
13,P_Q-4-295,6,LOC,St. Ulrich's and St. Afra's Abbey,http://dbpedia.org/resource/St._Ulrich's_and_S...,http://www.wikidata.org/entity/Q458378,"St. Ulrich's and St. Afra's Abbey, Augsburg",Benedictine abbey dedicated to Saint Ulrich an...,48.3614,10.9004


In [ ]:
#@title 2) Generate GeoJSON in the Linked Places format
#@markdown This format is consumed by Peripleo app.

from geojson import Feature, Point
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point as ShapelyPoint

def get_robust_bounds(geometries):
    """
    Computes a bounding box for a set of Shapely geometries,
    handling cases where geometries cross the antimeridian.

    Returns a bbox in the format [min_lon, min_lat, max_lon, max_lat].
    """
    if not geometries:
        return None

    # Get the simple min/max bounds first
    all_lon = [g.bounds[0] for g in geometries] + [g.bounds[2] for g in geometries]
    all_lat = [g.bounds[1] for g in geometries] + [g.bounds[3] for g in geometries]

    min_lon = min(all_lon)
    min_lat = min(all_lat)
    max_lon = max(all_lon)
    max_lat = max(all_lat)

    # Check for antimeridian crossing
    # If the span is > 180, the points must cross the antimeridian
    if (max_lon - min_lon) > 180:
        min_lon = -180
        max_lon = 180

    return [min_lon, min_lat, max_lon, max_lat]

def df2lp(df: pd.DataFrame) -> dict:

  features = []
  geometries = []

  # Ensure the DataFrame is sorted for correct grouping logic
  df_sorted = df.sort_values(by='wikidata_uri')

  for _, row in df_sorted.iterrows():
    if row.entity_label != "LOC" or pd.isna(row.wikidata_uri):
        continue
    if not row.wikidata_latitude or not row.wikidata_longitude:
        continue

    # Construct the GeoJSON Feature
    longitude = float(row.wikidata_longitude)
    latitude = float(row.wikidata_latitude)

    point = Point([longitude, latitude])

    uri = f'https://knowledgebase.sloanelab.org/resource/?uri=http%3A%2F%2Fsloanelab.org%2FE73%2Fbm_dataset%2F{row["Unique ID"]}'

    feature = Feature(
      geometry=point,
      properties={
        "title": row['Unique ID'],
        "description": "Description of the object",
        "url": f"https://www.britishmuseum.org/collection/object/{row['Unique ID']}"
      },
      # Simplified links for clarity
      links=[{
        "type": "seeAlso",
        "label": row['Unique ID'],
        "url": uri
      }]
    )
    feature["@id"] = uri
    features.append(feature)

    # Collect Shapely geometries for bounding box calculation
    geometries.append(ShapelyPoint(longitude, latitude))


  # Compute the bounding box
  bbox = get_robust_bounds(geometries)

  data = {
     "type": "FeatureCollection",
     "@context": "https://raw.githubusercontent.com/LinkedPasts/linked-places/master/linkedplaces-context-v1.1.jsonld",
     "features": features,
     "bbox": bbox
  }

  return data

# print(json.dumps(df2lp(df), indent=2))


In [ ]:
#@title Preliminary visualisation of Linked Places format data

# Import the necessary libraries
import folium
import json

# Assuming df2lp(df) is defined and returns a GeoJSON object
# with a bbox property.
geojson_data = df2lp(df)

# Check if a bbox exists in the GeoJSON data
if 'bbox' in geojson_data and geojson_data['bbox']:
    bbox = geojson_data['bbox']
    # Folium's fit_bounds() expects a list of [[min_lat, min_lon], [max_lat, max_lon]]
    folium_bounds = [[bbox[1], bbox[0]], [bbox[3], bbox[2]]]
    # 1. Initialize the map with the bounding box
    folium_map = folium.Map()
    folium_map.fit_bounds(folium_bounds)
else:
    # Fallback to a default location if no bbox is available
    folium_map = folium.Map(location=[51.509, -0.12], zoom_start=12)

# 2. Loop through each GeoJSON feature to add a marker
for feature in geojson_data['features']:
    # Extract properties and coordinates
    properties = feature['properties']
    coordinates = feature['geometry']['coordinates']

    # GeoJSON coordinates are [longitude, latitude]
    lat = coordinates[1]
    lon = coordinates[0]

    # Use an f-string to embed the title and URL into HTML
    popup_html = f"""
    <h3>{properties['title']}</h3>
    <p>{properties.get('description', 'No description available.')}</p>
    <a href="{properties['url']}" target="_blank">Visit website</a>
    """

    # Create the Folium popup object with the HTML
    popup = folium.Popup(popup_html, max_width=300)

    # Add a marker to the map
    folium.Marker(
        location=[lat, lon],
        popup=popup,
        tooltip=properties['title'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(folium_map)

# 3. Display the map
folium_map

#Create Secret

This provides permission for this notebook to access your Github account through Github's API.
Warning: be careful not to expose the secret value outside this Colab notebook.

* You must have a Github account (link)
* Go to https://github.com/settings/tokens
* On left go to Tokens classic
* Complete with any note
* Generate new token (classic) on dropdown
* Select repo tick box
* Generate token by clicking button at bottom
* Copy the token that is displayed

* Go to Colab - click key icon on left
* Add new secret
* Name: GITHUB_TOKEN, paste token value into Value
* Use Notebook Access toggle to authorise use in notebook


In [19]:
#@title 3) Make Peripleo app available on Github Pages
#@markdown **This script will perform a complete, automated setup for a publicly available Peripleo app with your data.**
#@markdown
#@markdown It will:
#@markdown 1. Create a new private repository by importing Peripleo.
#@markdown 2. Add your data and a configuration file.
#@markdown 3. Enable GitHub Pages to publish your site.
#@markdown
#@markdown This requires a GitHub Classic Personal Access Token with 'repo' scope, set as a Colab secret.

import os
import requests
import time
from github import Github, GithubException, Auth
import json
from google.colab import userdata

# --- 1. SET YOUR GITHUB REPOSITORY ---
GITHUB_USERNAME = "dianamaynard" #@param {type:"string"}
REPO_NAME = "test-peripleo-app" #@param {type:"string"}
SOURCE_REPO_OWNER = "britishlibrary"
SOURCE_REPO_NAME = "peripleo"

GEOJSON_FILE_PATH = "docs/data/historical_data.geojson"
CONFIG_FILE_PATH = "docs/peripleo.config.json"

# --- 2. ACCESS YOUR GITHUB TOKEN FROM COLAB SECRETS ---
try:
    github_token = userdata.get('GITHUB_TOKEN')
except KeyError:
    raise ValueError("GITHUB_TOKEN not found. Please set it as a Colab secret.")

# --- 3. PREPARE THE API REQUESTS ---
# Headers for the template import API call (requires special media type)
template_headers = {
    "Authorization": f"token {github_token}",
    "Accept": "application/vnd.github.baptiste-preview+json"
}

# Standard headers for all other authenticated API calls
standard_headers = {
    "Authorization": f"token {github_token}",
    "Accept": "application/vnd.github.v3+json"
}

repo_data = {
    "owner": GITHUB_USERNAME,
    "name": REPO_NAME,
    "description": "Peripleo map generated from Colab",
    "private": False
}
template_url = f"https://api.github.com/repos/{SOURCE_REPO_OWNER}/{SOURCE_REPO_NAME}/generate"

# --- 4. AUTHENTICATE WITH PYGITHUB ---
auth = Auth.Token(github_token)
g = Github(auth=auth)
user = g.get_user()
print(f"Authenticated as: {user.login}")

# --- 5. AUTOMATED WORKFLOW ---
print(f"--- Step 1: Importing '{SOURCE_REPO_NAME}' as a new private repository named '{REPO_NAME}' ---")
try:
    new_repo = user.get_repo(REPO_NAME)
    print(f"Repository '{REPO_NAME}' already exists under your account. Skipping import.")
except GithubException as e:
    if e.status == 404:
        print(f"Repository '{REPO_NAME}' not found. Importing from template now...")
        response = requests.post(template_url, headers=template_headers, json=repo_data)
        response.raise_for_status()

        repo_found = False
        while not repo_found:
            try:
                new_repo = user.get_repo(REPO_NAME)
                repo_found = True
            except GithubException:
                print("Waiting for repository creation...", end="\r")
                time.sleep(5)

        print(f"\nNew repository '{REPO_NAME}' created successfully at {new_repo.html_url}")

        # --- NEW: Programmatic check for template content ---
        print("\nWaiting for template files to be imported...")
        content_imported = False
        while not content_imported:
            try:
                # Check for the existence of the main index file as a proxy for completion
                new_repo.get_contents("docs/index.html")
                content_imported = True
            except GithubException:
                print("Waiting for template content...", end="\r")
                time.sleep(5)
        print("\nTemplate files have been successfully imported.")
    else:
        raise e

# --- 6. PREPARE YOUR GEOJSON DATA ---
# This is done in a previous cell

# --- 7. ADDING YOUR DATA TO THE REPOSITORY ---
print("\n--- Step 2: Adding your data to the repository ---")
# Create GeoJSON file content
geojson_content = json.dumps(geojson_data, indent=2)
geojson_commit_message = f"Automated update of {GEOJSON_FILE_PATH}"

try:
    contents = new_repo.get_contents(GEOJSON_FILE_PATH)
    new_repo.update_file(
        path=GEOJSON_FILE_PATH,
        message=geojson_commit_message,
        content=geojson_content,
        sha=contents.sha
    )
    print(f"Successfully updated {GEOJSON_FILE_PATH}.")
except GithubException as e:
    if e.status == 404:
        new_repo.create_file(
            path=GEOJSON_FILE_PATH,
            message=geojson_commit_message,
            content=geojson_content
        )
        print(f"Successfully created {GEOJSON_FILE_PATH}.")
    else:
        raise e

# Create or update the Peripleo config.json file
config_data = {
    "initial_bounds": geojson_data['bbox'],
    "map_style": "./map-style-OSM.json",
    "data": [
        {
            "name": "Disambiguation Test",
            "format": "LINKED_PLACES",
            "src": "./data/historical_data.geojson",
            "attribution": "Leiden Workshop"
        }
    ],
    "facets": [
        "type"
    ],
    "link_icons": [
        { "pattern": "maps.google.com",  "img": "./logos/maps.google.com.png", "label": "Google Maps" },
        { "pattern": "www.geograph.org.uk", "img": "./logos/geograph.org.png", "label": "Geograph" },
        { "pattern": "en.wikipedia.org", "img": "./logos/en.wikipedia.org.png", "label": "Wikipedia" },
        { "pattern": "www.wikidata.org", "img": None, "label": "Wikidata" },
        { "pattern": "www.geonames.org", "img": None, "label": "GeoNames" },
        { "pattern": "sws.geonames.org", "img": None, "label": "GeoNames" }
    ]
}

# Convert the Python dictionary to a JSON string with indentation for readability
config_content = json.dumps(config_data, indent=2)
config_commit_message = f"Automated update of {CONFIG_FILE_PATH}"
try:
    contents = new_repo.get_contents(CONFIG_FILE_PATH)
    new_repo.update_file(
        path=CONFIG_FILE_PATH,
        message=config_commit_message,
        content=config_content,
        sha=contents.sha
    )
    print(f"Successfully updated {CONFIG_FILE_PATH}.")
except GithubException as e:
    if e.status == 404:
        new_repo.create_file(
            path=CONFIG_FILE_PATH,
            message=config_commit_message,
            content=config_content
        )
        print(f"Successfully created {CONFIG_FILE_PATH}.")
    else:
        raise e


# --- 8. ENABLE GITHUB PAGES ---
print("\n--- Step 3: Enabling GitHub Pages ---")
pages_url = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPO_NAME}/pages"
pages_data = {
    "source": {
        "branch": "main",
        "path": "/docs"
    }
}
try:
    response = requests.post(pages_url, headers=standard_headers, json=pages_data)
    response.raise_for_status()

    # Wait for Pages to become active
    pages_active = False
    print(f"You can check the build status here: https://github.com/{GITHUB_USERNAME}/{REPO_NAME}/actions")
    while not pages_active:
        try:
            pages_status_url = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPO_NAME}/pages"
            pages_status = requests.get(pages_status_url, headers=standard_headers).json()
            if pages_status.get('status') == 'built':
                pages_active = True
                print("\nGitHub Pages enabled successfully.")
        except Exception:
            pass # Keep waiting
        print("Waiting for GitHub Pages to be provisioned...", end="\r")
        time.sleep(10)

    print(f"\nDeployment process complete!")
    print(f"Your repository is at: https://github.com/{GITHUB_USERNAME}/{REPO_NAME}")
    print(f"Your Peripleo map should be available shortly at: {pages_status.get('html_url')}")

except requests.exceptions.RequestException as e:
    print(f"Error enabling GitHub Pages: {e}. You may need to enable it manually.")

Authenticated as: dianamaynard
--- Step 1: Importing 'peripleo' as a new private repository named 'test-peripleo-app' ---
Repository 'test-peripleo-app' already exists under your account. Skipping import.

--- Step 2: Adding your data to the repository ---
Successfully updated docs/data/historical_data.geojson.
Successfully updated docs/peripleo.config.json.

--- Step 3: Enabling GitHub Pages ---
You can check the build status here: https://github.com/dianamaynard/test-peripleo-app/actions

GitHub Pages enabled successfully.

Deployment process complete!
Your repository is at: https://github.com/dianamaynard/test-peripleo-app
Your Peripleo map should be available shortly at: https://dianamaynard.github.io/test-peripleo-app/


# Output Data

The file linked below contains a GeoJSON representation of the locations mentioned in our historical documents. GeoJSON is a standard format for representing geographic data with a simple structure of points, lines, and polygons.

In [ ]:
#@title Map Data Download

# Convert the GeoJSON object to a string
geojson_string = json.dumps(geojson_data, indent=2)

# Encode the GeoJSON string for the data URI
geojson_bytes = geojson_string.encode('utf-8')
encoded_geojson = base64.b64encode(geojson_bytes).decode('utf-8')

# Create the HTML for a styled button link
button_html = f"""
<a href="data:application/json;charset=utf-8;base64,{encoded_geojson}"
   download="historical_data.geojson"
   style="
     background-color: #4CAF50; /* Green */
     border: none;
     color: white;
     padding: 15px 32px;
     text-align: center;
     text-decoration: none;
     display: inline-block;
     font-size: 16px;
     margin: 4px 2px;
     cursor: pointer;
     border-radius: 8px;
   ">
  Download Historical Data (GeoJSON)
</a>
"""

# Display the button in the Colab output
display(HTML(button_html))

## Variations and alternatives

notes on ways in which this recipe be could be adapted or where and when alternative approaches should/could be used.